In [7]:
import cv2
import numpy as np
import os
import scipy.io as sio
from extract_random import extract_random
from extract_random import extract_rgb
from cvpr_compare import cvpr_compare
from random import randint
from matplotlib import pyplot as plt

In [8]:
def extract_color_descriptor(img):
    # Compute the average red, green, and blue values as a basic color descriptor
    R = np.mean(img[:, :, 2])  # Note: OpenCV uses BGR format
    G = np.mean(img[:, :, 1])
    B = np.mean(img[:, :, 0])
    return np.array([R, G, B])

In [10]:
DATASET_FOLDER = "MSRC_ObjCategImageDatabase_v2"
OUT_FOLDER = "descriptors"
OUT_SUBFOLDER = 'globalRGBhisto'
DESCRIPTOR_FOLDER = 'descriptors'
DESCRIPTOR_SUBFOLDER = 'globalRGBhisto'
def create_descriptor_dir(out_folder, out_subfolder):
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
        os.makedirs(os.path.join(out_folder, out_subfolder), exist_ok=True)

# Extract Image Descriptors from the Dataset
def extract_descriptors(dataset_path, descriptor_path):
   for filename in os.listdir(os.path.join(dataset_path, 'Images')):
    if filename.endswith(".bmp"):
        img_path = os.path.join(dataset_path, 'Images', filename)
        img = cv2.imread(img_path).astype(np.float64) / 255.0  # Normalize the image
        fout = os.path.join(descriptor_path, filename).replace('.bmp', '.npy')
        # Call extractRandom (or another feature extraction function) to get the descriptor
        F = extract_rgb(img)        
        np.save(fout, F)

def 

In [5]:


# Load all descriptors
ALLFEAT = []
ALLFILES = []
for filename in os.listdir(os.path.join(DESCRIPTOR_FOLDER, DESCRIPTOR_SUBFOLDER)):
    if filename.endswith('.mat'):
        descriptor_path = os.path.join(DESCRIPTOR_FOLDER, DESCRIPTOR_SUBFOLDER, filename)
        img_path = os.path.join(DATASET_FOLDER, 'Images', filename.replace('.mat', '.bmp'))
        descriptor_data = sio.loadmat(descriptor_path)
        ALLFILES.append(img_path)
        ALLFEAT.append(descriptor_data['F'][0])  # Assuming F is a 1D array
# Convert ALLFEAT to a numpy array
ALLFEAT = np.array(ALLFEAT)

# Pick a random image as the query
NIMG = ALLFEAT.shape[0]
queryimg = randint(0, NIMG - 1)

# Display the query image
query_image_path = ALLFILES[queryimg]
query_image = cv2.imread(query_image_path)

if query_image is None:
    print(f"Failed to load query image at {query_image_path}")
else:
    cv2.imshow("Query Image", query_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Compute the distance between the query and all other descriptors
dst = []
query = ALLFEAT[queryimg]
for i in range(NIMG):
    candidate = ALLFEAT[i]
    distance = cvpr_compare(query, candidate)
    dst.append((distance, i))

# Sort the distances
dst.sort(key=lambda x: x[0])

2024-10-11 21:39:46.292 python[19895:4131275] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-11 21:39:46.292 python[19895:4131275] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [6]:
# Show the top 15 results
SHOW = 15
images = [] 
target_height = 100

for i in range(min(SHOW, len(dst))):
    img_path = ALLFILES[dst[i][1]]
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Failed to load image at {img_path}")
        continue
    
    # Calculate the new width to maintain the aspect ratio
    aspect_ratio = img.shape[1] / img.shape[0]
    new_width = int(target_height * aspect_ratio)
    
    # Resize the image to the target height while maintaining the aspect ratio
    img = cv2.resize(img, (new_width, target_height))
    images.append(img)

# Concatenate images horizontally
if images:
    concatenated_image = np.hstack(images)
    cv2.imshow("Results", concatenated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
else:
    print("No images to display.")